In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import feather
import re
from zopilote.keepLog import write_csv_log
from unidecode import unidecode

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.mode.chained_assignment = None

%matplotlib inline

In [2]:
sancionados = pd.read_csv('input_data/Proveedores y Contratistas Sancionados.csv')
print(sancionados.shape)
sancionados.head()
# El número de expediente no tiene el mismo formato que el código de expediente

(1504, 11)


,Proveedor o Contratista,Dependencia,Número de Expediente,Hechos de la Irregularidad,Objeto Social,Sentido de Resolución,Fecha de Notificación,Fecha de Resolución,Plazo años - meses,Monto,Nombre del Responsable de la información
0,"3R INNOVACIÓN Y DISEÑO, S.A. DE C.V.",INSTITUTO MEXICANO DEL SEGURO SOCIAL,0041/2013,LA EMPRESA POR CAUSAS IMPUTABLES A ELLA NO FOR...,3RINNOVACION@LIVE.COM.MX.,SANCIONATORIA CON MULTA,8/19/2014,8/6/2014,NaN,90664,LIC. FEDERICO DE ALBA MARTÍNEZ
1,"A AND P INTERNATIONAL, S.A. DE C.V.",PEMEX EXPLORACIÓN Y PRODUCCION,0014/2012,EXHIBIO INFORMACION FALSA CONSISTENTE EN: CERT...,INFORMATICA Y COMUNICACIONES,SANCIONATORIA CON MULTA E INHABILITACIÓN,5/7/2014,5/6/2014,1 AÑO,1,NaN
2,"A Y M CONSTRUCTORA, S.A. DE C.V.",SECRETARIA DE COMUNICACIONES Y TRANSPORTES,0121/2006,PRESENTÓ INFORMACIÓN FALSA RESPECTO A ENCONTRA...,NaN,SANCIONATORIA CON MULTA E INHABILITACIÓN,4/5/2007,3/23/2007,3 MESES,72540,JORGE TRUJILLO ABARCA
3,"A Y V INDUSTRIAL Y COMERCIAL, S.A. DE C.V.","III SERVICIOS, S. A. DE C. V.",0001/2013,PRESENTACION DE DOCUMENTACION E INFORMACION FALSA,"DISEÑAR, PROYECTAR, PLANEAR, ORGANIZAR, ASESOR...",SANCIONATORIA CON MULTA E INHABILITACIÓN,5/8/2014,1759-59-41,3 MESES,133502.74,JOSE ISMAEL JACOBO RODRIGUEZ
4,"A+D ARQUITECTOS, S.A. DE C.V.",COMISION FEDERAL DE ELECTRICIDAD,0187/2006,PRESENTÓ INFORMACIÓN FALSA RESPECTO A ENCONTRA...,DISEÑO ARQUITECTÓNICO Y CONSTRUCCIÓN,SANCIONATORIA CON MULTA E INHABILITACIÓN,11/6/2006,11/3/2006,3 MESES,75438.5,MIGUEL ANGEL PEREZ MAR


In [3]:
sancionados['año'] = sancionados['Número de Expediente         '].map(lambda x: x.split('/')[-1])

In [4]:
sancionados.año.value_counts()

2006    259
2010    184
2011    144
2009    139
2012    130
2013    127
2014    127
2007    120
2008    114
2005     66
2004     40
2015     28
2003     25
2001      1
Name: año, dtype: int64

In [5]:
sancionados['Sentido de Resolución '].value_counts()

SANCIONATORIA CON MULTA E INHABILITACIÓN    1166
SANCIONATORIA CON MULTA                      281
SANCIONATORIA                                 32
ABSOLUTORIA                                    1
Name: Sentido de Resolución , dtype: int64

In [6]:
valid_proceds = pd.read_csv('generated_data/procedimientos_validos_importe_pesos.csv',
                            dtype={'IDENTIFICADOR_CM':str, 'CLAVE_PROGRAMA':str, 'ANUNCIO':str, 'SIGLAS_PAIS':str,
                                   'ORGANISMO':str})

valid_proceds.head()

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS,CLAVEUC
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,208993,N44 MOB ETC,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23 00:00:00,NaN,2012-10-23 00:00:00,2012-12-31 00:00:00,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,512930.40,901024986
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,208992,N44 MOB ETC,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23 00:00:00,NaN,2012-10-23 00:00:00,2012-10-31 00:00:00,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,1887409.40,901024986
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,PAPELERIA CONSUMIBLES Y ACCESORIOS SA DE CV,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,257525,IEA.DJ.CONT. 1140/2012,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19 00:00:00,NaN,2012-11-12 00:00:00,2012-12-31 00:00:00,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,1892535.82,901024986
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761.0,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,NaN,ADJUDICACION DIRECTA FEDERAL,241054,CONSUMIBLES,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-09-06 08:49:00,2012-09-07 09:00:00,NaN,NaN,2012-09-11 00:00:00,2012-12-31 00:00:00,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07. Adjudicación Directa Nacional Ar...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,24234.00,901024986
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761.0,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,NaN,ADJUDICACION DIRECTA FEDERAL,474294,ADQUISICION DE EQUIPO DE COMPUTO,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-12-12 11:38:00,2012-12-14 10:00:00,NaN,NaN,2012-12-17 00:00:00,2012-12-31 00:00:00,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07. Adjudicación Directa Nacional Ar...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,18700.00,901024986


In [7]:
# Homologamos proveedores
valid_proceds['homologa_provs'] = valid_proceds.PROVEEDOR_CONTRATISTA.map(
    lambda x: unidecode(str(x)).lower().replace('"', '').replace('.', '').replace(',','').strip())

sancionados['homologa_provs'] = sancionados['Proveedor o Contratista'].map(
    lambda x: unidecode(str(x)).lower().replace('"', '').replace('.', '').replace(',','').strip())

In [8]:
sancionados.homologa_provs.value_counts().head()

mantenimiento construcciones y desarrollo industrial sa de cv    10
c y j construcciones sa de cv                                     7
continental serv sa de cv                                         6
mundo medico del norte sa de cv                                   6
conversion e impresion lopez ortiz sa de cv                       6
Name: homologa_provs, dtype: int64

In [9]:
sancionados.homologa_provs.nunique(), sancionados['Proveedor o Contratista'].nunique()

(1267, 1324)

In [10]:
valid_proceds[valid_proceds.homologa_provs.isin(sancionados.homologa_provs)].homologa_provs.nunique()

252

In [24]:
# No parece que podamos hacer una unión entre los sancionados y los procedimientos
valid_proceds[valid_proceds.homologa_provs.isin(sancionados.homologa_provs)]

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS,homologa_provs
35,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,94222.0,RODOLFO SEGURA SALAS,HABILITADO,PEQUENA,NaN,MXN,LA-901024986-N3-2012,Electrónica,LICITACION PUBLICA,780222,ADQUISICION DE UTENSILIOS,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-05-16 10:12:00,2012-05-28 09:00:00,2012-05-29 00:00:00,2012-06-08 00:00:00,2012-06-08 00:00:00,2012-07-31 00:00:00,182078,EQUIPAMIENTO COCINAS Y COMEDORES,Z15122015 01. Licitación Pública Nacional LAASSP,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,2.574900e+04,rodolfo segura salas
115,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-OFICINA DE ADQUISICIONES DE LA DELEGACION...,019GYR027,JOSE FRANCISCO MENDOZA MARTINEZ,NO MIPYME,NaN,VACUFARM SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-019GYR027-T19-2012,Electrónica,LICITACION PUBLICA,553628,CONSUMIBLES PARA EQUIPO MEDICO E INSTRUMENTAL ...,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-05-31 02:00:00,2012-06-20 09:00:00,2012-07-02 00:00:00,2012-07-17 00:00:00,2012-07-02 00:00:00,2012-12-31 00:00:00,187228,"ADQUISICION DE CONSUMIBLES PARA EQUIPO MEDICO,...",Z15122015 02. Licitación Pública Internacional...,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,2.830884e+04,vacufarm sa de cv
131,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-OFICINA DE ADQUISICIONES DE LA DELEGACION...,019GYR027,JOSE FRANCISCO MENDOZA MARTINEZ,NO MIPYME,NaN,ALBEK DE MEXICO SA DE CV,HABILITADO,MEDIANA,NaN,MXN,LA-019GYR027-N4-2012,Electrónica,LICITACION PUBLICA,880129,"ADQUISICION DE MEDICAMENTO, MATERIAL DE CURACI...",NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-01-31 20:36:00,2012-02-16 10:00:00,2012-02-20 00:00:00,2012-03-05 00:00:00,2012-03-05 00:00:00,2012-12-31 00:00:00,128913,"ADQUISICION DE MEDICAMENTO, MATERIAL DE CURACI...",Z15122015 01. Licitación Pública Nacional LAASSP,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,1.873188e+04,albek de mexico sa de cv
146,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-OFICINA DE ADQUISICIONES DE LA DELEGACION...,019GYR027,JOSE FRANCISCO MENDOZA MARTINEZ,MICRO,NaN,BIOMEDICA RONA SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-019GYR027-N6-2012,Electrónica,LICITACION PUBLICA,508604,MANTENIMIENTO A EQUIPO MEDICO,NaN,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-02-03 17:27:00,2012-02-17 10:00:00,2012-02-22 00:00:00,2012-03-06 00:00:00,2012-03-01 00:00:00,2012-12-31 00:00:00,130034,MANTENIMIENTO A EQUIPOS MEDICOS,Z15122015 01. Licitación Pública Nacional LAASSP,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,6.250000e+04,biomedica rona sa de cv
509,APF,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,SCT,SCT-CENTRO SCT SAN LUIS POTOSI #009000966,009000966,LUIS CARRION LARA,MICRO,NaN,ALYAMA SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-009000966-N116-2011,Mixta,LICITACION PUBLICA,433661,UNIFORMES,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2011-12-27 02:00:00,2012-01-13 09:30:00,2012-01-18 00:00:00,NaN,2012-02-03 00:00:00,2012-12-31 00:00:00,118996,"ADQUISICION DE UN LOTE DE UNIFORMES, CHAMARRAS...",Z15122015 01. Licitación Pública Nacional LAASSP,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,N

In [46]:
# Empresas sancionadas en 2015
sanc_15 = sancionados[sancionados.año=='2015']
sanc_15.homologa_provs.value_counts().head()

constructora metalmecanica y civil sa de cv     3
constructora el crizol sa de cv                 1
formula llantas sa de cv                        1
bioelectronica e instrumentos rafir sa de cv    1
global fulla internacional sa de cv             1
Name: homologa_provs, dtype: int64

In [48]:
sanc_15[sanc_15.homologa_provs=='constructora metalmecanica y civil sa de cv']

,Proveedor o Contratista,Dependencia,Número de Expediente,Hechos de la Irregularidad,Objeto Social,Sentido de Resolución,Fecha de Notificación,Fecha de Resolución,Plazo años - meses,Monto,Nombre del Responsable de la información,homologa_provs,año
332,"CONSTRUCTORA METALMECÁNICA Y CIVIL, S.A. DE C.V.",PEMEX REFINACION,0005/2015,"PRESUNTAMENTE PROPORCIONÓ INFORMACIÓN FALSA, D...",NaN,SANCIONATORIA CON MULTA E INHABILITACIÓN,9/17/2015,9/17/2015,1 AÑO,1,JORGE LUIS MEJIA ALONZO,constructora metalmecanica y civil sa de cv,2015
333,"CONSTRUCTORA METALMECÁNICA Y CIVIL, S.A. DE C.V.",PEMEX REFINACION,0006/2015,PRESUNTAMENTE PROPORCIONÓ INFORMACIÓN FALSA EN...,NaN,SANCIONATORIA CON MULTA E INHABILITACIÓN,9/17/2015,9/17/2015,1 AÑO,1,JORGE LUIS MEJIA ALONZO,constructora metalmecanica y civil sa de cv,2015
334,"CONSTRUCTORA METALMECÁNICA Y CIVIL, S.A. DE C.V.",PEMEX REFINACION,0007/2015,PRESUNTAMENTE PROPORCIONÓ INFORMACIÓN FALSA EN...,NaN,SANCIONATORIA CON MULTA E INHABILITACIÓN,9/17/2015,9/17/2015,1 AÑO,1,JORGE LUIS MEJIA ALONZO,constructora metalmecanica y civil sa de cv,2015


In [53]:
sancionados.Dependencia.value_counts().head()

INSTITUTO MEXICANO DEL SEGURO SOCIAL    217
SECRETARIA DE LA FUNCIÓN PÚBLICA        149
COMISION FEDERAL DE ELECTRICIDAD        131
PEMEX EXPLORACIÓN Y PRODUCCION          119
PEMEX PETROQUIMICA                       88
Name: Dependencia, dtype: int64

In [57]:
# Ver si vale la pena hacer join por año y proveedor
valid_proceds['año'] = valid_proceds.NUMERO_PROCEDIMIENTO.map(lambda x: x.split('-')[-1] if type(x)==str else x)

proceds_sancs = valid_proceds.merge(sancionados[['año', 'homologa_provs', 'Dependencia']], on=['año', 'homologa_provs'])

In [59]:
# No parece haber una forma de filtrar el procedimiento que causó la sanción
proceds_sancs

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS,homologa_provs,año,Dependencia
0,APF,COORDINACION NACIONAL DE PROSPERA PROGRAMA DE ...,PROSPERA,PROSPERA-PROSPERA-OFICINAS CENTRALES #020G00001,020G00001,MARIA DEL PILAR ORTIZ CARRASCO,MICRO,7557.0,IBERBYTE SA DE CV,HABILITADO,MICRO,NaN,MXN,IA-020G00001-T17-2012,Mixta,INVITACION A CUANDO MENOS 3 PERSONAS,121941,PEDIDO 075 2012,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-04-11 12:58:00,2012-04-30 09:00:00,2012-05-08 00:00:00,2012-05-08 00:00:00,2012-05-07 00:00:00,2012-06-07 00:00:00,168566,INSUMOS INFORMATICOS 2012,Z15122015 05. Invitación a cuando menos tres I...,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,252625.56,iberbyte sa de cv,2012,INSTITUTO MEXICANO DEL SEGURO SOCIAL
1,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-DEPARTAMENTO DE ADQUISICIONES DE LA DELEG...,019GYR011,CESAR AUGUSTO JIMENEZ FLORES,PEQUENA,7557.0,IBERBYTE SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-019GYR011-I55-2012,Mixta,LICITACION PUBLICA,164726,D120088,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-03-08 07:25:00,2012-03-28 10:00:00,2012-04-04 00:00:00,2012-04-17 00:00:00,2012-04-16 00:00:00,2012-12-31 00:00:00,146776,LA-019GYR011-I55-2012 ADQUISICION DE PAPELERI...,Z15122015 03. Licitación Pública Internacional...,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,0.0,225032.00,iberbyte sa de cv,2012,INSTITUTO MEXICANO DEL SEGURO SOCIAL
2,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-OFICINA DE ADQUISICION DE BIENES Y CONTRA...,019GYR008,JOSE ANDRES MARTINEZ AGUILAR,PEQUENA,7557.0,IBERBYTE SA DE CV,HABILITADO,MICRO,NaN,MXN,SA-019GYR008-N34-2012,Mixta,ADJUDICACION DIRECTA FEDERAL,115436,CONSUMIBLES EQUIPO DE COMPUTO,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-03-16 15:08:00,2012-03-26 11:00:00,NaN,2012-04-09 00:00:00,2012-04-08 00:00:00,2012-12-31 00:00:00,157823,"SA-019GYR008-N34-2012 PAPELERIA, UTILES DE OFI...",Z15122015 12. Adjudicación Directa Nacional Si...,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,114344.86,iberbyte sa de cv,2012,INSTITUTO MEXICANO DEL SEGURO SOCIAL
3,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-OFICINA DE ADQUISICION DE BIENES Y CONTRA...,019GYR028,DULCE MARIA REYES CASTRO,PEQUENA,7557.0,IBERBYTE SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-019GYR028-N25-2012,Mixta,LICITACION PUBLICA,847777,"LA-019GYR028-N25-2012 PAPELERIA, ARTICULOS DE ...",NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-02-28 02:00:00,2012-03-14 10:00:00,2012-03-30 00:00:00,NaN,2012-04-13 00:00:00,2012-12-31 00:00:00,141155,"LA-019GYR028-N25-2012 PAPELERIA, ARTICULOS DE ...",Z15122015 01. Licitación Pública Nacional LAASSP,NaN,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,NaN,NaN,703895.30,iberbyte sa de cv,2012,INSTITUTO MEXICANO DEL SEGURO SOCIAL
4,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-DEPARTAMENTO DE ADQUISICION DE BIENES Y C...,019GYR006,SERGIO MARTINEZ CEDILLO,PEQUENA,7557.0,IBERBYTE SA DE CV,HABILITADO,MICRO,NaN,MXN,SA-019GYR006-N159-2012,Presencial,ADJUDICACION DIRECTA FEDERAL,831330,PEDIDO CONSUMIBLES DE EQUIPO COMPUTO,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaN,NaN,NaN,NaN,2012-03-23 00:00:00,2012-03-26 00:00:00,156738,ADJ-052-12 CONSUMIBLES DE EQUIPO DE COMPUTO,Z15122015 12. Adjudicación Directa Nacional Si...